In [1]:
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder

In [2]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)
        #nn.BatchNorm2d(96)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        #nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        #nn.BatchNorm2d(384)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        nn.BatchNorm2d(384)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
        #nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.fc1 = nn.Linear(9216, 4096)
        nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 4096)
        #self.fc3 = nn.Linear(4096, 1000)
        self.fc3 = nn.Linear(4096, 29)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool3(F.relu(self.conv5(x)))
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        x = self.fc3(x)
        return x

In [3]:
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
train_data_dir = 'assets/asl/asl_alphabet_train'
test_data_dir = 'assets/asl/asl_alphabet_test'

training_data = ImageFolder(root=train_data_dir, transform=transform)
testing_data = ImageFolder(root=test_data_dir, transform=transform)

num_images_per_class = 500
subset_indices = []
for class_idx in range(len(training_data.classes)):
    indices = [idx for idx, (_, label_idx) in enumerate(training_data.samples) if label_idx == class_idx]
    selected_indices = random.sample(indices, num_images_per_class)  # Randomly select indices
    subset_indices.extend(selected_indices)

# Create a subset of the dataset with the selected indices
train_subset = Subset(training_data, subset_indices)

batch_size = 128
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [8]:
model = AlexNet()
num_epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        print(loss)
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {correct / total}")

tensor(3.3662, grad_fn=<NllLossBackward0>)
tensor(3.3680, grad_fn=<NllLossBackward0>)
tensor(3.3673, grad_fn=<NllLossBackward0>)
tensor(3.3671, grad_fn=<NllLossBackward0>)
tensor(3.3671, grad_fn=<NllLossBackward0>)
tensor(3.3667, grad_fn=<NllLossBackward0>)
tensor(3.3689, grad_fn=<NllLossBackward0>)
tensor(3.3677, grad_fn=<NllLossBackward0>)
tensor(3.3659, grad_fn=<NllLossBackward0>)
tensor(3.3663, grad_fn=<NllLossBackward0>)
tensor(3.3686, grad_fn=<NllLossBackward0>)
tensor(3.3675, grad_fn=<NllLossBackward0>)
tensor(3.3677, grad_fn=<NllLossBackward0>)
tensor(3.3691, grad_fn=<NllLossBackward0>)
tensor(3.3680, grad_fn=<NllLossBackward0>)
tensor(3.3684, grad_fn=<NllLossBackward0>)
tensor(3.3672, grad_fn=<NllLossBackward0>)
tensor(3.3675, grad_fn=<NllLossBackward0>)
tensor(3.3676, grad_fn=<NllLossBackward0>)
tensor(3.3677, grad_fn=<NllLossBackward0>)
tensor(3.3669, grad_fn=<NllLossBackward0>)
tensor(3.3687, grad_fn=<NllLossBackward0>)
tensor(3.3672, grad_fn=<NllLossBackward0>)
tensor(3.36

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000250F854BF50>>
Traceback (most recent call last):
  File "C:\Users\JeannotMunganga\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 785, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\JeannotMunganga\anaconda3\envs\Machine Learning PG\Lib\threading.py", line 1501, in enumerate
    def enumerate():
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
SAVED_MODEL_PATH = "./alex_net_sign.pth"
torch.save(model.state_dict(), SAVED_MODEL_PATH)